In [ ]:
import pandas as pd
with open("../collatz_conjecture_100m_data.txt","r",encoding='utf-8') as data:
    data.seek(0,2)
    len=data.tell
    
    x=li[0:-2]
    print(x)

2, 


In [ ]:
import pandas as pd
with open("../collatz_conjecture_100m_data.txt","r",encoding='utf-8') as data:
    batch_1m=[0]*1000000
    pan_data=pd.DataFrame()
    batch_1m[0]=1
    i=1
    x=0
    for line in data:
        line=line.strip()
        if len(line)<=1:continue
        if line[-1]==",":line=line[0:-2]
        batch_1m[i]=line
        if (i+1)%1000000==0:
            pan_data[f"start_num : {(i-1000000+2)+1000000*x}~{i+1+1000000*x}"]=batch_1m
            batch_1m=[0]*1000000
            i=-1
            x+=1
        i+=1
print(pan_data.shape)
print(pan_data[-1])

In [19]:
pan_data.iloc[2]

start_num : -1~1000000    98000004,49000002,24500001,73500004,36750002,1...
Name: 2, dtype: object

In [ ]:
# Parquet 저장 (압축 zstd 권장)
pan_data.to_parquet("dataset.parquet", engine="pyarrow", compression="zstd")

^C


In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq

# Parquet writer 열기
writer = None

with open("../collatz_conjecture_100m_data.txt", "r", encoding="utf-8") as f:
    batch_size = 1000000  # 100만 줄 단위로 처리
    lines = []
    for i, line in enumerate(f, 1):
        line=line.strip()
        if len(line)<=1:continue
        if line[-1]==",":line=line[-2]
        lines.append(line)
        if i % batch_size == 0:
            table = pa.Table.from_arrays([pa.array(lines)], names=[f"numbers"])
            if writer is None:
                writer = pq.ParquetWriter(f"collatz_100m.parquet", table.schema, compression="zstd")
            writer.write_table(table)
            lines = []  # 메모리 비우기

    if lines:
        table = pa.Table.from_arrays([pa.array(lines)], names=["numbers"])
        writer.write_table(table)

if writer:
    writer.close()


In [ ]:
import pyarrow.parquet as pq

table = pq.read_table("collatz_100m.parquet")
print("Row groups:", table.num_row_groups)

: 

In [ ]:
for i in range(parquet_file.num_row_groups):
    table = parquet_file.read_row_group(i)
    df = table.to_pandas()
    print(f"=== Row group {i} ===")
    print(df.head())